<a href="https://colab.research.google.com/github/Jahnavi927/Metabridge/blob/main/Copy_of_Meta_FED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#MERGING FILES
import pandas as pd
import glob

csv_files = glob.glob("*.csv")

for file in csv_files:
    try:
        df = pd.read_csv(file, nrows=5)  # just read 5 rows to check columns
        print(f"{file} → {df.columns.tolist()}")
    except Exception as e:
        print(f"Error reading {file}: {e}")


diabetes_val.csv → ['EventDateTime', 'CGM', 'Subject', 'Label', 'Label_num']
heart_trainA.csv → ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target', 'Subject']
diabetes_trainA.csv → ['EventDateTime', 'CGM', 'Subject', 'Label', 'Label_num']
heart_trainB.csv → ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target', 'Subject']
processed.cleveland.csv → ['63.0', '1.0', '1.0.1', '145.0', '233.0', '1.0.2', '2.0', '150.0', '0.0', '2.3', '3.0', '0.0.1', '6.0', '0']
heart_test.csv → ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target', 'Subject']
diabetes_trainB.csv → ['EventDateTime', 'CGM', 'Subject', 'Label', 'Label_num']
AZT1D_CGM_Labeled.csv → ['EventDateTime', 'CGM', 'Subject', 'Label', 'Label_num']
diabetes_test.csv → ['EventDateTime', 'CGM', 'Subject', 'Label', 'Label_num']
heart_val.

In [ ]:
import pandas as pd
import numpy as np

# Load raw Cleveland data
df = pd.read_csv("/content/processed.cleveland.csv", header=None)

# Assign headers
headers = [
    "age", "sex", "cp", "trestbps", "chol", "fbs",
    "restecg", "thalach", "exang", "oldpeak", "slope",
    "ca", "thal", "target"
]
df.columns = headers

# Replace '?' with NaN
df = df.replace('?', np.nan)

# Convert all columns to numeric
for col in df.columns:
    df[col] = pd.to_numeric(df[col])

# Handle missing values (choose one method)
# Option 1: Drop rows with missing values
df = df.dropna()
# Option 2: Fill missing values with median (alternative)
# df = df.fillna(df.median())

# Save cleaned CSV
df.to_csv("cleveland_preprocessed.csv", index=False)
print("✅ Preprocessed dataset saved with headers and missing values handled in 'cleveland_preprocessed.csv'")


✅ Preprocessed dataset saved with headers and missing values handled in 'cleveland_preprocessed.csv'


In [ ]:
#FINAL PRE-PROCESSING OF DIABETIS TIME-SERIES DATA
import pandas as pd
import numpy as np

# Load merged dataset
df = pd.read_csv("AZT1D_CGM_Labeled.csv")

# Keep only the important columns
df = df[["EventDateTime", "CGM", "Subject"]]

# Drop rows with missing values
df = df.dropna(subset=["EventDateTime", "CGM"])

# Convert CGM to numeric (coerce errors to NaN)
df["CGM"] = pd.to_numeric(df["CGM"], errors='coerce')

# Fill or drop remaining NaNs in CGM
df = df.dropna(subset=["CGM"])  # or use df["CGM"].fillna(df["CGM"].median(), inplace=True)

# Optionally, sort by Subject and time
df = df.sort_values(by=["Subject", "EventDateTime"]).reset_index(drop=True)

# Save the cleaned dataset
df.to_csv("AZT1D_CGM_Time_CGM_Subject.csv", index=False)

print("✅ Cleaned dataset saved as AZT1D_CGM_Time_CGM_Subject.csv")
print("Final shape:", df.shape)
print("Columns:", df.columns.tolist())
print("Unique subjects:", df['Subject'].nunique())
print(df["Subject"].unique())

✅ Cleaned dataset saved as AZT1D_CGM_Time_CGM_Subject.csv
Final shape: (591532, 3)
Columns: ['EventDateTime', 'CGM', 'Subject']
Unique subjects: 25
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25]


In [ ]:
import pandas as pd

# Load dataset
df = pd.read_csv("AZT1D_CGM_Labeled.csv")

# Keep only the important columns
df = df[["EventDateTime", "CGM", "Subject"]]

# Convert EventDateTime to datetime objects
df["EventDateTime"] = pd.to_datetime(df["EventDateTime"], errors="coerce")

# Drop rows with invalid datetime or missing CGM
df = df.dropna(subset=["EventDateTime", "CGM"])

# Sort data by Subject and EventDateTime
df = df.sort_values(by=["Subject", "EventDateTime"]).reset_index(drop=True)
# Drop duplicate rows
df = df.drop_duplicates(subset=["EventDateTime", "CGM", "Subject"])

# Add Hypo / Normal / Hyper labels
def label_glucose(value):
    if value < 70:
        return "Hypo"
    elif value > 180:
        return "Hyper"
    else:
        return "Normal"

df["Label"] = df["CGM"].apply(label_glucose)
#convert  labels to binary numbers
label_map = {"Hypo": 1, "Normal": 0, "Hyper": 2}
df["Label_num"] = df["Label"].map(label_map)
# Check if any duplicates are still present
duplicates = df.duplicated(subset=["EventDateTime", "CGM", "Subject"]).sum()
# check if exists any duplicate rows
if duplicates == 0:
    print("No duplicate rows remain")
else:
    print(f"Found {duplicates} duplicate rows")

# Save the cleaned + labeled dataset
df.to_csv("AZT1D_CGM_Labeled.csv", index=False)

print("Cleaned & labeled dataset saved as AZT1D_CGM_Labeled.csv")
print("Final shape:", df.shape)
print("Columns:", df.columns.tolist())
print("Unique subjects:", df['Subject'].nunique())

No duplicate rows remain
Cleaned & labeled dataset saved as AZT1D_CGM_Labeled.csv
Final shape: (591532, 5)
Columns: ['EventDateTime', 'CGM', 'Subject', 'Label', 'Label_num']
Unique subjects: 25


In [ ]:
#APPLYING MEDIUM SHIFTS
import pandas as pd
import numpy as np
import os

# -----------------------------
# Load datasets
# -----------------------------
cleveland_path = "cleveland_preprocessed.csv"
aztid_path = "AZT1D_CGM_Labeled.csv"

if not os.path.exists(cleveland_path):
    raise FileNotFoundError("Cleveland dataset not found. Please preprocess and save as cleveland_preprocessed.csv")
if not os.path.exists(aztid_path):
    raise FileNotFoundError("AZT1D dataset not found. Please preprocess and save as AZT1D_CGM_Labeled.csv")

cleveland_df = pd.read_csv(cleveland_path)
aztid_df = pd.read_csv(aztid_path)


# Helper functions

def simulate_label_shift_multiclass(df, target_col="target", intensity="medium"):
    shifted = df.copy()
    n = len(shifted)

    frac_map = {"mild": 0.1, "medium": 0.3, "strong": 0.5}
    frac = frac_map[intensity]

    n_shift = int(n * frac)
    idx = np.random.choice(shifted.index, size=n_shift, replace=False)

    classes = shifted[target_col].unique()
    new_labels = np.random.choice(classes, size=n_shift)
    shifted.loc[idx, target_col] = new_labels

    return shifted

def simulate_feature_shift(df, feature_cols, intensity="medium"):
    shifted = df.copy()

    std_map = {"mild": 0.5, "medium": 2.0, "strong": 5.0}
    noise_std = std_map[intensity]

    for col in feature_cols:
        if pd.api.types.is_numeric_dtype(shifted[col]):
            n_shift = int(len(shifted) * 0.3)  # shift 30% of rows
            idx = np.random.choice(shifted.index, size=n_shift, replace=False)
            noise = np.random.normal(0, noise_std, n_shift)
            shifted.loc[idx, col] = shifted.loc[idx, col] + noise

    return shifted


# Apply shifts for Cleveland

cleveland_versions = {}
for intensity in ["mild", "medium", "strong"]:
    df_label_shift = simulate_label_shift_multiclass(cleveland_df, target_col="target", intensity=intensity)
    df_feature_shift = simulate_feature_shift(
        df_label_shift,
        feature_cols=[c for c in cleveland_df.columns if c != "target"],
        intensity=intensity
    )
    cleveland_versions[intensity] = df_feature_shift
    df_feature_shift.to_csv(f"cleveland_shifted_{intensity}.csv", index=False)


# Apply shifts for AZT1D

aztid_versions = {}
for intensity in ["mild", "medium", "strong"]:
    df_shift = simulate_feature_shift(aztid_df, feature_cols=["CGM"], intensity=intensity)
    aztid_versions[intensity] = df_shift
    df_shift.to_csv(f"aztid_shifted_{intensity}.csv", index=False)

# Print BEFORE vs AFTER summaries

print("\n Shifts created successfully!\n")

# Cleveland target distribution
print("Cleveland target distributions:")
before_cleveland = cleveland_df["target"].value_counts(normalize=True)
print("\nBefore:\n", before_cleveland)

for intensity, df in cleveland_versions.items():
    after = df["target"].value_counts(normalize=True)
    print(f"\nAfter ({intensity.capitalize()}):\n", after)

# AZT1D CGM stats
print("\nAZT1D CGM statistics:")
before_aztid = aztid_df["CGM"].describe().round(2)
print("\nBefore:\n", before_aztid)

for intensity, df in aztid_versions.items():
    after = df["CGM"].describe().round(2)
    print(f"\nAfter ({intensity.capitalize()}):\n", after)


 Shifts created successfully!

Cleveland target distributions:

Before:
 target
0    0.538721
1    0.181818
2    0.117845
3    0.117845
4    0.043771
Name: proportion, dtype: float64

After (Mild):
 target
0    0.508418
1    0.198653
3    0.117845
2    0.111111
4    0.063973
Name: proportion, dtype: float64

After (Medium):
 target
0    0.420875
1    0.185185
2    0.161616
3    0.131313
4    0.101010
Name: proportion, dtype: float64

After (Strong):
 target
0    0.397306
1    0.188552
2    0.148148
3    0.138047
4    0.127946
Name: proportion, dtype: float64

AZT1D CGM statistics:

Before:
 count    591532.00
mean        146.54
std          47.74
min          40.00
25%         114.00
50%         138.00
75%         171.00
max         400.00
Name: CGM, dtype: float64

After (Mild):
 count    591532.00
mean        146.54
std          47.74
min          39.14
25%         113.56
50%         137.73
75%         171.15
max         400.62
Name: CGM, dtype: float64

After (Medium):
 count    59

In [ ]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split

# ------------------------------
# Step 1: Load datasets
# ------------------------------
heart_data = pd.read_csv("cleveland_shifted_medium.csv")
diab_data = pd.read_csv("aztid_shifted_medium.csv")

# If heart dataset has no 'Subject', create one
if "Subject" not in heart_data.columns:
    heart_data["Subject"] = heart_data.index

# ------------------------------
# Step 2: Split global train/val/test
# ------------------------------
# Heart
heart_train, heart_temp = train_test_split(
    heart_data, test_size=0.3, random_state=42, stratify=heart_data["target"]
)
heart_val, heart_test = train_test_split(
    heart_temp, test_size=0.5, random_state=42, stratify=heart_temp["target"]
)

# Diabetes (participant-wise)
def split_timeseries(df, participant_col="Subject", frac_train=0.7, frac_val=0.15):
    train_list, val_list, test_list = [], [], []
    for pid, group in df.groupby(participant_col):
        n = len(group)
        n_train = int(n * frac_train)
        n_val = int(n * frac_val)
        train_list.append(group.iloc[:n_train])
        val_list.append(group.iloc[n_train:n_train+n_val])
        test_list.append(group.iloc[n_train+n_val:])
    return pd.concat(train_list), pd.concat(val_list), pd.concat(test_list)

diab_train, diab_val, diab_test = split_timeseries(diab_data)

# ------------------------------
# Step 3: Create crossed federations
# ------------------------------
# Federation A: 70% heart + 30% diabetes
fedA_heart_train = heart_train.sample(frac=0.7, random_state=42)
fedB_heart_train = heart_train.drop(fedA_heart_train.index)

fedA_diab_train = diab_train.sample(frac=0.3, random_state=42)
fedB_diab_train = diab_train.drop(fedA_diab_train.index)

# ------------------------------
# Step 4: Create folders
# ------------------------------
os.makedirs("data_federations/federationA", exist_ok=True)
os.makedirs("data_federations/federationB", exist_ok=True)
os.makedirs("data_federations/validation", exist_ok=True)
os.makedirs("data_federations/testing", exist_ok=True)

# ------------------------------
# Step 5: Save CSVs
# ------------------------------
fedA_heart_train.to_csv("data_federations/federationA/heart_train.csv", index=False)
fedA_diab_train.to_csv("data_federations/federationA/diabetes_train.csv", index=False)

fedB_heart_train.to_csv("data_federations/federationB/heart_train.csv", index=False)
fedB_diab_train.to_csv("data_federations/federationB/diabetes_train.csv", index=False)

# Global validation
heart_val.to_csv("data_federations/validation/heart_val.csv", index=False)
diab_val.to_csv("data_federations/validation/diabetes_val.csv", index=False)

# Global testing
heart_test.to_csv("data_federations/testing/heart_test.csv", index=False)
diab_test.to_csv("data_federations/testing/diabetes_test.csv", index=False)

# ------------------------------
# Step 6: Summary
# ------------------------------
print("✅ Crossed federated datasets created!")
print(f"Federation A: Heart {len(fedA_heart_train)} rows (~70%), Diabetes {len(fedA_diab_train)} rows (~30%)")
print(f"Federation B: Heart {len(fedB_heart_train)} rows (~30%), Diabetes {len(fedB_diab_train)} rows (~70%)")
print(f"Global Validation: Heart {len(heart_val)}, Diabetes {len(diab_val)}")
print(f"Global Testing: Heart {len(heart_test)}, Diabetes {len(diab_test)}")



✅ Crossed federated datasets created!
Federation A: Heart 145 rows (~70%), Diabetes 124218 rows (~30%)
Federation B: Heart 62 rows (~30%), Diabetes 289843 rows (~70%)
Global Validation: Heart 45, Diabetes 88719
Global Testing: Heart 45, Diabetes 88752


In [ ]:
# ========================= Metafed CKD — Training =========================
# Model: GRU (seq) + Transformer (seq) + MLP (tabular) → two heads (Heart-5, Diabetes-3)
# Stages: Local → CKD cycles → Personalization
# Artifacts saved to: ./metafed_ckd_artifacts_final/
# =================================================================================
import os, time, copy, math, random
from typing import Dict, Tuple

import numpy as np
import pandas as pd
import joblib

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import StratifiedShuffleSplit


DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ARTIFACT_DIR = "metafed_ckd_artifacts_final"
os.makedirs(ARTIFACT_DIR, exist_ok=True)

# input files
A_HEART_CSV = "data_federations/federationA/heart_train.csv"
A_DIAB_CSV  = "data_federations/federationA/diabetes_train.csv"
B_HEART_CSV = "data_federations/federationB/heart_train.csv"
B_DIAB_CSV  = "data_federations/federationB/diabetes_train.csv"

# training hyperparams
LOCAL_EPOCHS = 8
CKD_CYCLES = 3
CKD_EPOCHS_PER_CYCLE = 1
PERSONAL_EPOCHS = 6
PATIENCE = 3
VAL_SPLIT = 0.15

BATCH_HEART = 64
BATCH_DIAB  = 256

LR_BASE = 1e-3
LR_CKD  = 5e-4
LR_PERS = 3e-4
WEIGHT_DECAY = 1e-6
CLIP_GRAD = 1.0

# KD
KD_TEMPERATURE = 2.5
KD_WEIGHT = 0.7

# diabetes trend labeling
DIAB_SEQ_IN = 30
DIAB_SLOPE_WINDOW = 12
SLOPE_UP = 0.5
SLOPE_DOWN = -0.5

# model sizes
GRU_HIDDEN = 128
GRU_LAYERS = 1
TRANS_DMODEL = 128
TRANS_NHEAD = 4
TRANS_LAYERS = 2
DROPOUT = 0.25
HEART_HIDDEN = [128, 64]
DIAB_HEAD_HIDDEN = 64
LABEL_SMOOTH = 0.05

# seeds
SEED = 42
def set_seed(seed=SEED):
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
set_seed()


# Utilities

def compute_trend_labels(df: pd.DataFrame, subj_col="Subject", value_col="CGM"):
    """Adds CGM_scaled and trend (0=Flat,1=Up,2=Down) using slope on per-subject scaled series."""
    df = df.copy()
    df[value_col] = pd.to_numeric(df[value_col], errors="coerce")
    df = df.dropna(subset=[value_col]).reset_index(drop=True)

    sub_stats = {}
    df["CGM_scaled"] = np.nan
    df["trend"] = -1
    for subj, g in df.groupby(subj_col):
        g = g.sort_values("EventDateTime").reset_index()
        vals = g[value_col].values.astype(np.float32)
        mu = vals.mean()
        sd = vals.std() if vals.std() > 1e-6 else 1.0
        sub_stats[subj] = (float(mu), float(sd))
        scaled = (vals - mu) / sd
        df.loc[g["index"], "CGM_scaled"] = scaled

        n = DIAB_SLOPE_WINDOW
        for i in range(len(scaled)):
            if i < n:
                df.loc[g["index"][i], "trend"] = 0
            else:
                slope = (scaled[i] - scaled[i-n]) / float(n)
                if slope > SLOPE_UP:      lab = 1
                elif slope < SLOPE_DOWN:  lab = 2
                else:                     lab = 0
                df.loc[g["index"][i], "trend"] = lab

    df = df.dropna(subset=["CGM_scaled"]).reset_index(drop=True)
    df["trend"] = df["trend"].astype(int)
    return df, sub_stats

def make_class_weights(y_int: np.ndarray, num_classes: int = 5, power: float = 0.5) -> torch.Tensor:
    """Inverse-frequency class weights with smoothing power to avoid extremes."""
    counts = np.bincount(y_int, minlength=num_classes).astype(np.float32)
    counts[counts == 0] = 1.0
    inv = (1.0 / counts) ** power
    w = inv / inv.mean()
    return torch.tensor(w, dtype=torch.float32)

# ----------------------------
# Datasets
# ----------------------------
class HeartDataset(Dataset):
    def __init__(self, df: pd.DataFrame, scaler: StandardScaler = None, fit_scaler=False):
        self.df = df.reset_index(drop=True).copy()
        X = self.df.drop(columns=["target"], errors="ignore")
        if scaler is None and fit_scaler:
            self.scaler = StandardScaler().fit(X)
            Xs = self.scaler.transform(X)
        elif scaler is not None:
            self.scaler = scaler
            Xs = self.scaler.transform(X)
        else:
            self.scaler = None
            Xs = X.values
        self.X = torch.tensor(Xs, dtype=torch.float32)
        self.y = torch.tensor(self.df["target"].astype(int).values, dtype=torch.long)
    def __len__(self): return len(self.df)
    def __getitem__(self, idx): return self.X[idx], self.y[idx]

class DiabetesSeqDataset(Dataset):
    def __init__(self, df: pd.DataFrame, seq_in=DIAB_SEQ_IN):
        self.samples = []
        df = df.copy()
        for _, g in df.groupby("Subject"):
            g = g.sort_values("EventDateTime")
            arr = g["CGM_scaled"].values.astype(np.float32)
            trends = g["trend"].astype(int).values
            for i in range(len(arr) - seq_in):
                x = arr[i:i+seq_in]
                y = trends[i+seq_in]
                self.samples.append((x, int(y)))
    def __len__(self): return len(self.samples)
    def __getitem__(self, idx):
        x, y = self.samples[idx]
        return torch.tensor(x, dtype=torch.float32).unsqueeze(-1), torch.tensor(y, dtype=torch.long)

# Model
class HybridGRUTransformerMTL(nn.Module):
    def __init__(self, heart_feat_dim: int):
        super().__init__()
        # sequence path (for diabetes)
        self.gru = nn.GRU(input_size=1, hidden_size=GRU_HIDDEN, num_layers=GRU_LAYERS,
                          batch_first=True, bidirectional=True)
        self.proj = nn.Linear(GRU_HIDDEN * 2, TRANS_DMODEL)
        enc = nn.TransformerEncoderLayer(d_model=TRANS_DMODEL, nhead=TRANS_NHEAD,
                                         dim_feedforward=TRANS_DMODEL*2, batch_first=True, dropout=DROPOUT)
        self.transformer = nn.TransformerEncoder(enc, num_layers=TRANS_LAYERS)
        self.seq_norm = nn.LayerNorm(TRANS_DMODEL)

        # tabular path (for heart)
        self.heart_mlp = nn.Sequential(
            nn.Linear(heart_feat_dim, HEART_HIDDEN[0]),
            nn.BatchNorm1d(HEART_HIDDEN[0]),
            nn.ReLU(),
            nn.Dropout(DROPOUT),
            nn.Linear(HEART_HIDDEN[0], HEART_HIDDEN[1]),
            nn.BatchNorm1d(HEART_HIDDEN[1]),
            nn.ReLU(),
        )

        # fusion + heads
        fused = HEART_HIDDEN[1] + TRANS_DMODEL
        self.combined_proj = nn.Sequential(
            nn.Linear(fused, fused // 2),
            nn.ReLU()
        )
        self.heart_head = nn.Sequential(
            nn.Linear(fused // 2, 64),
            nn.ReLU(),
            nn.Dropout(DROPOUT),
            nn.Linear(64, 5)   # 5 classes
        )
        self.diab_head = nn.Sequential(
            nn.Linear(TRANS_DMODEL, DIAB_HEAD_HIDDEN),
            nn.ReLU(),
            nn.Dropout(DROPOUT),
            nn.Linear(DIAB_HEAD_HIDDEN, 3)  # 3 classes
        )

    def forward(self, heart_x, diab_x):
        # diabetes sequence → GRU → Transformer → last token
        g, _ = self.gru(diab_x)       # [B, T, 2H]
        p = self.proj(g)              # [B, T, D]
        t = self.transformer(p)       # [B, T, D]
        t = self.seq_norm(t)
        diab_emb = t[:, -1, :]        # [B, D]
        diab_logits = self.diab_head(diab_emb)

        # heart tabular → MLP → fuse with diabetes context (broadcast mean if batch-size mismatch)
        heart_feat = self.heart_mlp(heart_x)
        if heart_x.size(0) == diab_x.size(0):
            context = diab_emb
        else:
            context = diab_emb.mean(dim=0, keepdim=True).expand(heart_x.size(0), -1)
        fused = torch.cat([heart_feat, context], dim=-1)
        z = self.combined_proj(fused)
        heart_logits = self.heart_head(z)
        return heart_logits, diab_logits


# Losses / KD

def ce_with_label_smoothing(logits, targets, class_weights=None, smoothing=0.0):
    # PyTorch CE supports label_smoothing natively
    return F.cross_entropy(logits, targets, weight=class_weights, label_smoothing=smoothing)

def kd_loss(student_logits, teacher_logits, T):
    s = F.log_softmax(student_logits / T, dim=-1)
    t = F.softmax(teacher_logits / T, dim=-1)
    return F.kl_div(s, t, reduction="batchmean") * (T * T)


# Local training (per client)

def train_local_one_client(heart_csv, diab_csv, out_prefix, epochs=LOCAL_EPOCHS) -> Dict:
    print(f"\n=== LOCAL TRAINING @ {out_prefix} ===")
    # heart
    heart_df = pd.read_csv(heart_csv).reset_index(drop=True)
    heart_scaler = StandardScaler().fit(heart_df.drop(columns=["target"]))
    heart_ds = HeartDataset(heart_df, scaler=heart_scaler, fit_scaler=False)

    # imbalance handling
    y_np = heart_df["target"].astype(int).values
    cls_w = make_class_weights(y_np, num_classes=5).to(DEVICE)
    # weighted sampler increases minority sampling
    class_counts = np.bincount(y_np, minlength=5).astype(np.float32)
    samp_weights = (1.0 / class_counts)[y_np]
    sampler = WeightedRandomSampler(weights=samp_weights, num_samples=len(y_np), replacement=True)
    heart_loader = DataLoader(heart_ds, batch_size=BATCH_HEART, sampler=sampler, drop_last=False)

    # diabetes
    diab_df_raw = pd.read_csv(diab_csv)
    diab_df, sub_stats = compute_trend_labels(diab_df_raw)
    diab_ds = DiabetesSeqDataset(diab_df, seq_in=DIAB_SEQ_IN)
    diab_loader = DataLoader(diab_ds, batch_size=BATCH_DIAB, shuffle=True, drop_last=False)

    heart_dim = heart_df.drop(columns=["target"]).shape[1]
    model = HybridGRUTransformerMTL(heart_feat_dim=heart_dim).to(DEVICE)

    opt = torch.optim.Adam(model.parameters(), lr=LR_BASE, weight_decay=WEIGHT_DECAY)

    model.train()
    steps = max(len(heart_loader), len(diab_loader))
    h_iter = iter(heart_loader); d_iter = iter(diab_loader)
    for ep in range(1, epochs+1):
        run_h = 0.0; run_d = 0.0
        for _ in range(steps):
            try:    hx, hy = next(h_iter)
            except StopIteration: h_iter = iter(heart_loader); hx, hy = next(h_iter)
            try:    dx, dy = next(d_iter)
            except StopIteration: d_iter = iter(diab_loader); dx, dy = next(d_iter)
            hx, hy, dx, dy = hx.to(DEVICE), hy.to(DEVICE), dx.to(DEVICE), dy.to(DEVICE)

            opt.zero_grad()
            log_h, log_d = model(hx, dx)
            lh = ce_with_label_smoothing(log_h, hy, class_weights=cls_w, smoothing=LABEL_SMOOTH)
            ld = F.cross_entropy(log_d, dy)
            loss = lh + ld
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), CLIP_GRAD)
            opt.step()
            run_h += lh.item(); run_d += ld.item()

        print(f"Client {out_prefix} ep {ep}/{epochs} | heart_loss {run_h/steps:.4f} | diab_loss {run_d/steps:.4f}")



    # save artifacts
    state = {k: v.cpu() for k, v in model.state_dict().items()}
    torch.save(state, os.path.join(ARTIFACT_DIR, f"local_{out_prefix}_state.pt"))
    joblib.dump(heart_scaler, os.path.join(ARTIFACT_DIR, f"scaler_heart_{out_prefix}.pkl"))
    joblib.dump(sub_stats,   os.path.join(ARTIFACT_DIR, f"sub_stats_{out_prefix}.pkl"))
    return {"state_dict": state, "heart_scaler": heart_scaler, "sub_stats": sub_stats, "heart_dim": heart_dim}

# CKD cycle

def run_ckd_cycle(teacher_art, student_art, s_heart_csv, s_diab_csv, out_prefix, epochs=CKD_EPOCHS_PER_CYCLE) -> Dict:
    print(f"\n--- CKD Teacher→Student ({out_prefix}) ---")
    heart_df = pd.read_csv(s_heart_csv)
    heart_scaler = student_art["heart_scaler"]
    heart_ds = HeartDataset(heart_df, scaler=heart_scaler, fit_scaler=False)

    # class weights (student's distribution)
    y_np = heart_df["target"].astype(int).values
    cls_w = make_class_weights(y_np, num_classes=5).to(DEVICE)
    # no need weighted sampler here; KD stabilizes; we still use simple shuffle loader
    heart_loader = DataLoader(heart_ds, batch_size=BATCH_HEART, shuffle=True, drop_last=False)

    diab_df_raw = pd.read_csv(s_diab_csv)
    diab_df, _ = compute_trend_labels(diab_df_raw)
    diab_ds = DiabetesSeqDataset(diab_df, seq_in=DIAB_SEQ_IN)
    diab_loader = DataLoader(diab_ds, batch_size=BATCH_DIAB, shuffle=True, drop_last=False)

    heart_dim = student_art["heart_dim"]
    teacher = HybridGRUTransformerMTL(heart_feat_dim=heart_dim).to(DEVICE)
    student = HybridGRUTransformerMTL(heart_feat_dim=heart_dim).to(DEVICE)
    teacher.load_state_dict(student_art["state_dict"])
    student.load_state_dict(student_art["state_dict"])

    teacher.eval()
    student.train()
    opt = torch.optim.Adam(student.parameters(), lr=LR_CKD, weight_decay=WEIGHT_DECAY)

    steps = max(len(heart_loader), len(diab_loader))
    h_iter = iter(heart_loader); d_iter = iter(diab_loader)
    for ep in range(1, epochs+1):
        run = 0.0
        for _ in range(steps):
            try:    hx, hy = next(h_iter)
            except StopIteration: h_iter = iter(heart_loader); hx, hy = next(h_iter)
            try:    dx, dy = next(d_iter)
            except StopIteration: d_iter = iter(diab_loader); dx, dy = next(d_iter)
            hx, hy, dx, dy = hx.to(DEVICE), hy.to(DEVICE), dx.to(DEVICE), dy.to(DEVICE)

            with torch.no_grad():
                t_h, t_d = teacher(hx, dx)

            s_h, s_d = student(hx, dx)
            hard_h = ce_with_label_smoothing(s_h, hy, class_weights=cls_w, smoothing=LABEL_SMOOTH)
            hard_d = F.cross_entropy(s_d, dy)
            kd_h = kd_loss(s_h, t_h, KD_TEMPERATURE)
            kd_d = kd_loss(s_d, t_d, KD_TEMPERATURE)
            loss = (1-KD_WEIGHT)*(hard_h + hard_d) + KD_WEIGHT*(kd_h + kd_d)

            opt.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(student.parameters(), CLIP_GRAD)
            opt.step()
            run += loss.item()
        print(f" CKD ep {ep}/{epochs} | loss {run/steps:.4f}")

    state = {k: v.cpu() for k, v in student.state_dict().items()}
    return {"state_dict": state, "heart_scaler": heart_scaler, "sub_stats": student_art["sub_stats"], "heart_dim": heart_dim}

# Personalization with val split + early stopping (heart-first objective)

def personalize_client(artifact, heart_csv, diab_csv, out_prefix, epochs=PERSONAL_EPOCHS, patience=PATIENCE):
    print(f"\n=== PERSONALIZING {out_prefix} ===")
    heart_df = pd.read_csv(heart_csv).reset_index(drop=True)
    scaler = artifact["heart_scaler"]

    # stratified split
    y = heart_df["target"].astype(int).values
    sss = StratifiedShuffleSplit(n_splits=1, test_size=VAL_SPLIT, random_state=SEED)
    train_idx, val_idx = next(sss.split(heart_df, y))
    heart_tr = heart_df.iloc[train_idx].reset_index(drop=True)
    heart_va = heart_df.iloc[val_idx].reset_index(drop=True)

    # datasets & loaders
    ds_tr = HeartDataset(heart_tr, scaler=scaler, fit_scaler=False)
    ds_va = HeartDataset(heart_va, scaler=scaler, fit_scaler=False)
    # keep sampler in personalization too
    y_tr = heart_tr["target"].astype(int).values
    cls_w = make_class_weights(y_tr, 5).to(DEVICE)
    class_counts = np.bincount(y_tr, minlength=5).astype(np.float32); class_counts[class_counts==0]=1
    samp_w = (1.0 / class_counts)[y_tr]
    sampler = WeightedRandomSampler(weights=samp_w, num_samples=len(y_tr), replacement=True)
    dl_tr = DataLoader(ds_tr, batch_size=BATCH_HEART, sampler=sampler)
    dl_va = DataLoader(ds_va, batch_size=256, shuffle=False)

    # diabetes for joint stability
    diab_df_raw = pd.read_csv(diab_csv)
    diab_df, _ = compute_trend_labels(diab_df_raw)
    diab_ds = DiabetesSeqDataset(diab_df, seq_in=DIAB_SEQ_IN)
    dl_diab = DataLoader(diab_ds, batch_size=BATCH_DIAB, shuffle=True)

    heart_dim = artifact["heart_dim"]
    model = HybridGRUTransformerMTL(heart_feat_dim=heart_dim).to(DEVICE)
    model.load_state_dict(artifact["state_dict"])

    # freeze → unfreeze selected
    for p in model.parameters(): p.requires_grad = False
    for p in model.proj.parameters(): p.requires_grad = True
    for p in model.transformer.parameters(): p.requires_grad = True
    for p in model.heart_mlp.parameters(): p.requires_grad = True
    for p in model.combined_proj.parameters(): p.requires_grad = True
    for p in model.heart_head.parameters(): p.requires_grad = True
    for p in model.diab_head.parameters(): p.requires_grad = True

    params = [p for p in model.parameters() if p.requires_grad]
    opt = torch.optim.Adam(params, lr=LR_PERS, weight_decay=WEIGHT_DECAY)

    best_f1 = -1.0
    best_state = copy.deepcopy(model.state_dict())
    no_improve = 0

    steps = max(len(dl_tr), len(dl_diab))
    h_iter = iter(dl_tr); d_iter = iter(dl_diab)
    for ep in range(1, epochs+1):
        model.train()
        run_h = 0.0; run_d = 0.0
        for _ in range(steps):
            try:    hx, hy = next(h_iter)
            except StopIteration: h_iter = iter(dl_tr); hx, hy = next(h_iter)
            try:    dx, dy = next(d_iter)
            except StopIteration: d_iter = iter(dl_diab); dx, dy = next(d_iter)
            hx, hy, dx, dy = hx.to(DEVICE), hy.to(DEVICE), dx.to(DEVICE), dy.to(DEVICE)

            opt.zero_grad()
            log_h, log_d = model(hx, dx)
            lh = ce_with_label_smoothing(log_h, hy, class_weights=cls_w, smoothing=LABEL_SMOOTH)
            ld = F.cross_entropy(log_d, dy)
            loss = lh + 0.5*ld
            loss.backward()
            torch.nn.utils.clip_grad_norm_(params, CLIP_GRAD)
            opt.step()
            run_h += lh.item(); run_d += ld.item()

        # validate (heart only)
        model.eval()
        preds, gold = [], []
        with torch.no_grad():
            for xb, yb in dl_va:
                xb = xb.to(DEVICE)
                logits, _ = model(xb, torch.zeros((xb.size(0), DIAB_SEQ_IN, 1), device=DEVICE))
                preds.append(logits.argmax(1).cpu().numpy()); gold.append(yb.numpy())
        preds = np.concatenate(preds); gold = np.concatenate(gold)
        f1 = f1_score(gold, preds, average="macro")
        print(f" Personalize ep {ep}/{epochs} | heart_loss {run_h/steps:.4f} | diab_loss {run_d/steps:.4f} | val_f1 {f1:.4f}")

        if f1 > best_f1:
            best_f1 = f1
            best_state = copy.deepcopy(model.state_dict())
            no_improve = 0
        else:
            no_improve += 1
            if no_improve >= patience:
                print(" Early stopping triggered.")
                break

    # load best & save
    model.load_state_dict(best_state)
    out_path = os.path.join(ARTIFACT_DIR, f"personalized_{out_prefix}_state.pt")
    torch.save({k: v.cpu() for k, v in model.state_dict().items()}, out_path)
    # scalers & stats already saved by local step, but we keep separate aliases too:
    joblib.dump(artifact["heart_scaler"], os.path.join(ARTIFACT_DIR, f"personal_scaler_heart_{out_prefix}.pkl"))
    joblib.dump(artifact["sub_stats"],   os.path.join(ARTIFACT_DIR, f"personal_substats_{out_prefix}.pkl"))
    print(f"Saved: {out_path}")

    return {"state_dict": {k: v.cpu() for k, v in model.state_dict().items()},
            "heart_scaler": artifact["heart_scaler"],
            "sub_stats": artifact["sub_stats"],
            "heart_dim": heart_dim}

def run_training_pipeline():
    t0 = time.time()
    # Local training
    artA = train_local_one_client(A_HEART_CSV, A_DIAB_CSV, out_prefix="A", epochs=LOCAL_EPOCHS)
    artB = train_local_one_client(B_HEART_CSV, B_DIAB_CSV, out_prefix="B", epochs=LOCAL_EPOCHS)

    # CKD cycles (A↔B)
    studentA, studentB = artA, artB
    for cyc in range(1, CKD_CYCLES+1):
        print(f"\n========== CKD cycle {cyc}/{CKD_CYCLES} ==========")
        studentB = run_ckd_cycle(teacher_art=studentA, student_art=studentB,
                                 s_heart_csv=B_HEART_CSV, s_diab_csv=B_DIAB_CSV,
                                 out_prefix="B", epochs=CKD_EPOCHS_PER_CYCLE)
        torch.save(studentB["state_dict"], os.path.join(ARTIFACT_DIR, f"ckd_cycle{cyc}_B_state.pt"))
        studentA = run_ckd_cycle(teacher_art=studentB, student_art=studentA,
                                 s_heart_csv=A_HEART_CSV, s_diab_csv=A_DIAB_CSV,
                                 out_prefix="A", epochs=CKD_EPOCHS_PER_CYCLE)
        torch.save(studentA["state_dict"], os.path.join(ARTIFACT_DIR, f"ckd_cycle{cyc}_A_state.pt"))

    # Personalize (with early stopping)
    persA = personalize_client(studentA, A_HEART_CSV, A_DIAB_CSV, out_prefix="A", epochs=PERSONAL_EPOCHS, patience=PATIENCE)
    persB = personalize_client(studentB, B_HEART_CSV, B_DIAB_CSV, out_prefix="B", epochs=PERSONAL_EPOCHS, patience=PATIENCE)

    # final aliases (for easy loading later)
    torch.save(persA["state_dict"], os.path.join(ARTIFACT_DIR, "final_A_state.pt"))
    torch.save(persB["state_dict"], os.path.join(ARTIFACT_DIR, "final_B_state.pt"))
    joblib.dump(persA["heart_scaler"], os.path.join(ARTIFACT_DIR, "final_heart_scaler_A.pkl"))
    joblib.dump(persB["heart_scaler"], os.path.join(ARTIFACT_DIR, "final_heart_scaler_B.pkl"))
    joblib.dump(persA["sub_stats"], os.path.join(ARTIFACT_DIR, "final_sub_stats_A.pkl"))
    joblib.dump(persB["sub_stats"], os.path.join(ARTIFACT_DIR, "final_sub_stats_B.pkl"))

    print(f"\nAll stages complete. Artifacts → {ARTIFACT_DIR}")
    print("Elapsed (s):", time.time() - t0)


# RUN

if __name__ == "__main__":
    run_training_pipeline()



=== LOCAL TRAINING @ A ===
Client A ep 1/8 | heart_loss 0.6534 | diab_loss 0.0080
Client A ep 2/8 | heart_loss 0.3106 | diab_loss 0.0028
Client A ep 3/8 | heart_loss 0.2798 | diab_loss 0.0029
Client A ep 4/8 | heart_loss 0.2699 | diab_loss 0.0030
Client A ep 5/8 | heart_loss 0.2649 | diab_loss 0.0030
Client A ep 6/8 | heart_loss 0.2584 | diab_loss 0.0028
Client A ep 7/8 | heart_loss 0.2541 | diab_loss 0.0030
Client A ep 8/8 | heart_loss 0.2544 | diab_loss 0.0028

=== LOCAL TRAINING @ B ===
Client B ep 1/8 | heart_loss 0.3091 | diab_loss 0.0031
Client B ep 2/8 | heart_loss 0.2342 | diab_loss 0.0013
Client B ep 3/8 | heart_loss 0.2317 | diab_loss 0.0013
Client B ep 4/8 | heart_loss 0.2316 | diab_loss 0.0013
Client B ep 5/8 | heart_loss 0.2318 | diab_loss 0.0013
Client B ep 6/8 | heart_loss 0.2306 | diab_loss 0.0013
Client B ep 7/8 | heart_loss 0.2298 | diab_loss 0.0013
Client B ep 8/8 | heart_loss 0.2308 | diab_loss 0.0014

========== CKD cycle 1/3 ==========

--- CKD Teacher→Student (B

In [ ]:
import os
print(os.listdir("metafed_ckd_artifacts_final"))


['ckd_cycle2_A_state.pt', 'ckd_cycle3_A_state.pt', 'final_heart_scaler_A.pkl', 'ckd_cycle3_B_state.pt', 'local_B_state.pt', 'personalized_B_state.pt', 'sub_stats_A.pkl', 'personal_scaler_heart_B.pkl', 'ckd_cycle1_B_state.pt', 'scaler_heart_A.pkl', 'final_sub_stats_B.pkl', 'personal_substats_B.pkl', 'personalized_A_state.pt', 'personal_scaler_heart_A.pkl', 'final_heart_scaler_B.pkl', 'local_A_state.pt', 'final_B_state.pt', 'personal_substats_A.pkl', 'sub_stats_B.pkl', 'final_sub_stats_A.pkl', 'scaler_heart_B.pkl', 'final_A_state.pt', 'ckd_cycle2_B_state.pt', 'ckd_cycle1_A_state.pt']


In [ ]:
                    #TESTING METAFED.......

import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from torch.utils.data import Dataset, DataLoader
import torch
import joblib

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ----- PATHS -----
ART = "metafed_ckd_artifacts_final"
HEART_TEST = "heart_test_balanced.csv"
DIAB_TEST = "diabetes_test.csv"   # old diabetes test remains same

# ----- DATASET CLASSES -----
class HeartDataset(Dataset):
    def __init__(self, df, scaler):
        X = scaler.transform(df.drop(columns=["target"]))
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(df["target"].astype(int).values, dtype=torch.long)
    def __len__(self): return len(self.y)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

class DiabetesSeqTest(Dataset):
    def __init__(self, df, seq_len=30):
        self.xs = []
        self.ys = []
        df = df.sort_values("EventDateTime")
        arr = df["CGM_scaled"].values
        lab = df["trend"].values
        for i in range(len(arr)-seq_len):
            self.xs.append(arr[i:i+seq_len])
            self.ys.append(lab[i+seq_len])
        self.xs = torch.tensor(self.xs).float().unsqueeze(-1)
        self.ys = torch.tensor(self.ys).long()

    def __len__(self): return len(self.ys)
    def __getitem__(self, idx): return self.xs[idx], self.ys[idx]

# ----- PREP DIABETES DATA -----
def prepare_diabetes(df):
    df = df.copy()
    df["CGM"] = pd.to_numeric(df["CGM"], errors="coerce")
    df = df.dropna(subset=["CGM"]).reset_index(drop=True)

    df["Subject"] = df["Subject"].astype(str)
    df = df.sort_values(["Subject", "EventDateTime"]).reset_index(drop=True)

    df["CGM_scaled"] = df.groupby("Subject")["CGM"].transform(lambda x: (x - x.mean())/(x.std()+1e-6))

    slopes = []
    w = 12
    for subj, g in df.groupby("Subject"):
        s = g["CGM_scaled"].values
        tvals = []
        for i in range(len(s)):
            if i < w: tvals.append(0)
            else:
                sl = (s[i] - s[i-w]) / w
                tvals.append(1 if sl>0.5 else 2 if sl<-0.5 else 0)
        slopes.extend(tvals)

    df["trend"] = slopes
    return df

# ----- MAIN EVALUATION -----
def evaluate(prefix):
    print(f"\n[+] Evaluating Client {prefix}")

    # Load heart scaler
    scaler = joblib.load(f"{ART}/scaler_heart_{prefix}.pkl")

    # Load heart test + clean
    heart_test = pd.read_csv(HEART_TEST)
    heart_test = heart_test.replace("?", np.nan)
    heart_test = heart_test.apply(pd.to_numeric, errors="ignore")
    for col in heart_test.columns:
        if col != "target" and heart_test[col].dtype != object:
            heart_test[col] = heart_test[col].fillna(heart_test[col].mean())

    heart_ds = HeartDataset(heart_test, scaler)
    heart_loader = DataLoader(heart_ds, batch_size=64)

    # Load weights
    state = torch.load(f"{ART}/personalized_{prefix}_state.pt", map_location=DEVICE)

    # Build model
    feat_dim = heart_test.drop(columns=["target"]).shape[1]
    model = HybridGRUTransformerMTL(feat_dim).to(DEVICE)

    # ✅ FIX: allow mismatched keys
    model.load_state_dict(state, strict=False)
    model.eval()

    # ----- HEART PRED -----
    yh_true, yh_pred = [], []
    with torch.no_grad():
        for x, y in heart_loader:
            x = x.to(DEVICE)
            logits_h, _ = model(x, torch.zeros((x.size(0), 30, 1), device=DEVICE))
            yh_true.extend(y.numpy())
            yh_pred.extend(logits_h.argmax(dim=1).cpu().numpy())

    print("\n===== HEART TEST REPORT =====")
    print(classification_report(yh_true, yh_pred))


    # ----- DIABETES TEST -----
    diab_test = prepare_diabetes(pd.read_csv(DIAB_TEST))
    diab_ds = DiabetesSeqTest(diab_test)
    diab_loader = DataLoader(diab_ds, batch_size=256)

    dy_true, dy_pred = [], []
    with torch.no_grad():
        for x, y in diab_loader:
            x = x.to(DEVICE)
            _, logits_d = model(torch.zeros((x.size(0), feat_dim), device=DEVICE), x)
            dy_true.extend(y.numpy())
            dy_pred.extend(logits_d.argmax(dim=1).cpu().numpy())

    print("\n===== DIABETES TEST REPORT =====")
    print("Accuracy:", accuracy_score(dy_true, dy_pred))
    print("Macro F1:", f1_score(dy_true, dy_pred, average="macro"))

# Run both
evaluate("A")
evaluate("B")



[+] Evaluating Client A

===== HEART TEST REPORT =====
              precision    recall  f1-score   support

           0       0.23      0.32      0.27        19
           1       0.38      0.53      0.44        19
           2       0.00      0.00      0.00        19
           3       0.36      0.26      0.30        19
           4       0.45      0.47      0.46        19

    accuracy                           0.32        95
   macro avg       0.28      0.32      0.30        95
weighted avg       0.28      0.32      0.30        95



/tmp/ipython-input-209513802.py:79: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  heart_test = heart_test.apply(pd.to_numeric, errors="ignore")
/tmp/ipython-input-209513802.py:37: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  self.xs = torch.tensor(self.xs).float().unsqueeze(-1)



===== DIABETES TEST REPORT =====
Accuracy: 1.0
Macro F1: 1.0

[+] Evaluating Client B

===== HEART TEST REPORT =====
              precision    recall  f1-score   support

           0       0.29      0.42      0.34        19
           1       0.07      0.11      0.08        19
           2       0.00      0.00      0.00        19
           3       0.27      0.32      0.29        19
           4       0.62      0.26      0.37        19

    accuracy                           0.22        95
   macro avg       0.25      0.22      0.22        95
weighted avg       0.25      0.22      0.22        95



/tmp/ipython-input-209513802.py:79: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  heart_test = heart_test.apply(pd.to_numeric, errors="ignore")



===== DIABETES TEST REPORT =====
Accuracy: 1.0
Macro F1: 1.0


In [ ]:
import pandas as pd
import numpy as np
import torch
import joblib
from sklearn.metrics import accuracy_score, f1_score
from torch.utils.data import Dataset, DataLoader

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ART = "metafed_ckd_artifacts_final"
HEART_VAL = "heart_val_balanced.csv"
DIAB_VAL = "diabetes_val.csv"

# ---------------- HEART DATASET ----------------
class HeartDataset(Dataset):
    def __init__(self, df, scaler):
        X = scaler.transform(df.drop(columns=["target"]))
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(df["target"].astype(int).values, dtype=torch.long)
    def __len__(self): return len(self.y)
    def __getitem__(self, i): return self.X[i], self.y[i]

# ---------------- DIABETES SEQ DATASET ----------------
class DiabetesSeqVal(Dataset):
    def __init__(self, df, seq=30):
        xs, ys = [], []
        for _, g in df.groupby("Subject"):
            g = g.sort_values("EventDateTime")
            arr = g["CGM_scaled"].values
            lab = g["trend"].values
            for i in range(len(arr)-seq):
                xs.append(arr[i:i+seq])
                ys.append(lab[i+seq])
        self.X = torch.tensor(xs).float().unsqueeze(-1)
        self.Y = torch.tensor(ys).long()
    def __len__(self): return len(self.Y)
    def __getitem__(self, i): return self.X[i], self.Y[i]

# ---------------- DIABETES PREPROCESS ----------------
def prepare_diabetes(df):
    df = df.copy()
    df["CGM"] = pd.to_numeric(df["CGM"], errors="coerce")
    df = df.dropna(subset=["CGM"]).reset_index(drop=True)
    df["Subject"] = df["Subject"].astype(str)
    df = df.sort_values(["Subject", "EventDateTime"]).reset_index(drop=True)
    df["CGM_scaled"] = df.groupby("Subject")["CGM"].transform(lambda x: (x-x.mean())/(x.std()+1e-6))

    slopes = []
    w = 12
    for subj, g in df.groupby("Subject"):
        s = g["CGM_scaled"].values
        t = []
        for i in range(len(s)):
            if i < w: t.append(0)
            else:
                sl = (s[i]-s[i-w])/w
                t.append(1 if sl>0.5 else 2 if sl<-0.5 else 0)
        slopes.extend(t)
    df["trend"] = slopes
    return df

# ---------------- VALIDATION FUNCTION ----------------
def validate(prefix):
    print(f"\n----- VALIDATION: CLIENT {prefix} -----")

    scaler = joblib.load(f"{ART}/scaler_heart_{prefix}.pkl")
    state = torch.load(f"{ART}/personalized_{prefix}_state.pt", map_location=DEVICE)

    hv = pd.read_csv(HEART_VAL).replace("?", np.nan)
    hv = hv.apply(pd.to_numeric, errors="ignore").fillna(hv.mean())
    heart_ds = HeartDataset(hv, scaler)
    heart_loader = DataLoader(heart_ds, batch_size=64)

    feat_dim = hv.drop(columns=["target"]).shape[1]
    model = HybridGRUTransformerMTL(feat_dim).to(DEVICE)
    model.load_state_dict(state, strict=False)
    model.eval()

    # Heart
    yh_true, yh_pred = [], []
    with torch.no_grad():
        for x, y in heart_loader:
            x = x.to(DEVICE)
            logits, _ = model(x, torch.zeros((x.size(0), 30, 1), device=DEVICE))
            yh_true.extend(y.numpy())
            yh_pred.extend(logits.argmax(1).cpu().numpy())

    print(f"Heart Accuracy: {accuracy_score(yh_true, yh_pred):.4f}")

    # Diabetes
    dv = prepare_diabetes(pd.read_csv(DIAB_VAL))
    diab_ds = DiabetesSeqVal(dv)
    diab_loader = DataLoader(diab_ds, batch_size=256)

    dy_true, dy_pred = [], []
    with torch.no_grad():
        for x, y in diab_loader:
            x = x.to(DEVICE)
            _, logits = model(torch.zeros((x.size(0), feat_dim), device=DEVICE), x)
            dy_true.extend(y.numpy())
            dy_pred.extend(logits.argmax(1).cpu().numpy())

    print(f"Diabetes Accuracy: {accuracy_score(dy_true, dy_pred):.4f}")

# ---------------- RUN ----------------
validate("A")
validate("B")



----- VALIDATION: CLIENT A -----
Heart Accuracy: 0.5273


/tmp/ipython-input-60135791.py:70: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  hv = hv.apply(pd.to_numeric, errors="ignore").fillna(hv.mean())


Diabetes Accuracy: 1.0000

----- VALIDATION: CLIENT B -----
Heart Accuracy: 0.2909


/tmp/ipython-input-60135791.py:70: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  hv = hv.apply(pd.to_numeric, errors="ignore").fillna(hv.mean())


Diabetes Accuracy: 1.0000
